# Code

In [3]:
#!pip install berserk

In [91]:
import pandas as pd, numpy as np
from tqdm import tqdm
from numba import njit

pd.set_option("display.precision", 20)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 1000)

time: 0 ns (started: 2023-07-09 18:14:51 +03:00)


<IPython.core.display.Javascript object>

In [58]:
%load_ext autoreload
%load_ext autotime
%load_ext nb_black
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
time: 15 ms (started: 2023-07-09 17:53:50 +03:00)


<IPython.core.display.Javascript object>

In [56]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

time: 16 ms (started: 2023-07-09 17:53:48 +03:00)


<IPython.core.display.Javascript object>

In [2]:
import berserk

session = berserk.TokenSession('lip_8ARjS5a7gCQwXOTglCH3')
client = berserk.Client(session=session)

In [ ]:
weetik_games=list(client.games.export_by_player(username='Weetik',rated=True,perf_type="blitz",clocks=True,literate=True))

In [59]:
Semenychev_Alexander_games = list(client.games.export_by_player(username="Semenychev_Alexander", rated=True, perf_type="blitz", clocks=True, literate=True))

time: 11min 6s (started: 2023-07-09 17:54:04 +03:00)


<IPython.core.display.Javascript object>

In [60]:
len(weetik_games), len(Semenychev_Alexander_games)

(8541, 15307)

time: 15 ms (started: 2023-07-09 18:05:10 +03:00)


<IPython.core.display.Javascript object>

In [9]:
weetik_games[1]

{'id': 'VkpwTcuR',
 'rated': True,
 'variant': 'standard',
 'speed': 'blitz',
 'perf': 'blitz',
 'createdAt': datetime.datetime(2023, 6, 30, 21, 13, 40, 70000, tzinfo=datetime.timezone.utc),
 'lastMoveAt': datetime.datetime(2023, 6, 30, 21, 17, 55, 204000, tzinfo=datetime.timezone.utc),
 'status': 'outoftime',
 'players': {'white': {'user': {'name': 'Alphaperro', 'id': 'alphaperro'},
   'rating': 2381,
   'ratingDiff': 7},
  'black': {'user': {'name': 'Weetik', 'title': 'FM', 'id': 'weetik'},
   'rating': 2473,
   'ratingDiff': -8}},
 'winner': 'white',
 'moves': 'd4 d5 c4 e6 Nc3 Nf6 cxd5 exd5 e3 c6 Bd3 Bd6 Nge2 O-O f3 Re8 O-O Qc7 h3 Nh5 e4 Ng3 Nxg3 Bxg3 e5 Qe7 f4 Qh4 Qf3 h5 Ne2 c5 Qxg3 Qxg3 Nxg3 cxd4 Nxh5 Nc6 a3 a6 Bd2 b5 Rac1 Bb7 Ng3 g6 b4 Rec8 Rc5 a5 Bxb5 axb4 axb4 Nxb4 Bxb4 Rxc5 Bxc5 Ra5 Rb1 Bc6 Bxc6 Rxc5 Bb5 Rc3 Ne2',
 'clocks': [18003,
  18003,
  17915,
  17931,
  17851,
  17835,
  17771,
  17179,
  17659,
  16747,
  17595,
  16611,
  17267,
  16491,
  17203,
  16075,
  17139,
  

In [103]:
def get_streaks(games:list,target_player:str,draw_breaks_streak:bool=True)->pd.DataFrame():
    
    nstreaks=0
    opponents=set()
    tournaments=set()    
    titled_opponents=set()
    totdominance=0.0

    lines=[]

    for game in tqdm(games[::-1]):
        is_winner_again=False    
        if game['status'] in ('outoftime','timeout'): continue

        if game['status'] not in ('draw','stalemate'):
            winner=game['winner']
            if winner in ('white','black'):
                if game['players'][winner]['user']['name']==target_player:
                    is_winner_again=True
                    winner_rating= game['players'][winner]['rating']
        else:
            if not draw_breaks_streak: continue

        if is_winner_again:
            nstreaks+=1

            if nstreaks==1:
                first_game_id=game['id']
                first_game_dt=game['createdAt']

            opponent='white' if winner=='black' else 'black'
            opponent=game['players'][opponent]
            opponents.add(opponent['user']['name'])
            if opponent['user'].get('title'):
                titled_opponents.add(opponent['user'].get('title')+" "+opponent['user'].get('name'))
            tournament=game.get('tournament')
            if tournament:
                tournaments.add(tournament)
            opponent_rating= opponent['rating']
            totdominance+=float(opponent_rating)-float(winner_rating)
        else:
            if nstreaks>0:
                if not titled_opponents:
                    titled_opponents=""
                else:
                    titled_opponents=str(len(titled_opponents))+": "+','.join(titled_opponents)
                lines.append([nstreaks,titled_opponents,len(opponents),totdominance/nstreaks,','.join(tournaments),first_game_dt,first_game_id])


            nstreaks=0
            opponents=set()
            tournaments=set()    
            titled_opponents=set()
            totdominance=0.0
    
    df=pd.DataFrame(lines,columns='nwins ntitled nopponents avg_dominance tournament first_game_dt first_game_id'.split())
    df['first_game_dt']=df['first_game_dt'].dt.tz_localize(None)
    return df


time: 0 ns (started: 2023-07-09 18:17:37 +03:00)


<IPython.core.display.Javascript object>

In [104]:
weetik_streaks = get_streaks(games=weetik_games, target_player="Weetik", draw_breaks_streak=True)
weetik_streaks

100%|██████████████████████████████████████████████████████████████████████████| 8541/8541 [00:00<00:00, 356815.38it/s]


,nwins,ntitled,nopponents,avg_dominance,tournament,first_game_dt,first_game_id
0,7,,6,-53.42857142857143060155,4EqeuRo5,2016-12-22 20:19:16.355,2MHmXuyO
1,9,1: WGM maili15,8,-141.22222222222222853816,"4EqeuRo5,TqH4KwYw",2017-06-30 23:54:22.456,wvrZdJbN
2,23,1: IM kenta123,19,-275.08695652173912549188,"11y0CwCg,TqH4KwYw",2017-07-01 05:54:01.476,zbbFV6ND
3,1,,1,-21.00000000000000000000,,2017-09-19 20:37:28.501,uwEbRhV3
4,4,,2,-28.00000000000000000000,,2017-09-19 20:47:29.677,qNXPseZH
...,...,...,...,...,...,...,...
1239,12,1: NM kostya24,11,-368.33333333333331438553,"DwBjhvEJ,zp5YUnoq",2023-06-01 17:15:13.273,9Jhawype
1240,6,,6,-447.16666666666668561447,DwBjhvEJ,2023-06-29 17:19:27.628,QvhnEmpi
1241,1,,1,-1604.00000000000000000000,DwBjhvEJ,2023-06-29 17:57:08.918,CcjfzpnT
1242,1,,1,-74.00000000000000000000,,2023-06-30 18:43:55.518,5U1mm3bX


time: 47 ms (started: 2023-07-09 18:17:39 +03:00)


<IPython.core.display.Javascript object>

In [106]:
semenychev_streaks = get_streaks(games=Semenychev_Alexander_games, target_player="Semenychev_Alexander", draw_breaks_streak=False)
semenychev_streaks

100%|████████████████████████████████████████████████████████████████████████| 15307/15307 [00:00<00:00, 448145.44it/s]


,nwins,ntitled,nopponents,avg_dominance,tournament,first_game_dt,first_game_id
0,11,1: NM Lopez09,11,-39.81818181818182011966,,2020-11-06 06:34:01.001,ZArhLHIh
1,1,,1,25.00000000000000000000,,2020-11-06 08:26:04.382,JF7RTKqA
2,4,1: IM Romsta,4,-74.50000000000000000000,,2020-11-06 08:35:29.152,LCjOainU
3,2,"2: IM KfLoRiAn,IM Akali666",2,47.00000000000000000000,,2020-11-06 16:40:18.377,yEX3cG6o
4,2,1: GM GingerGM,2,-26.50000000000000000000,,2020-11-06 16:50:09.795,tZqBvRef
...,...,...,...,...,...,...,...
2416,5,,5,-264.80000000000001136868,7p0YW6ha,2023-07-07 19:09:39.893,RTMAOJc3
2417,1,,1,-179.00000000000000000000,7p0YW6ha,2023-07-08 17:40:50.546,ROXDbNx7
2418,2,,2,-71.50000000000000000000,7p0YW6ha,2023-07-08 18:11:40.142,pDUg3T0v
2419,2,,2,-127.00000000000000000000,7p0YW6ha,2023-07-08 18:36:00.155,pU8xqN1o


time: 47 ms (started: 2023-07-09 18:17:42 +03:00)


<IPython.core.display.Javascript object>

# Results

In [105]:
weetik_streaks.query("nwins>=13")

,nwins,ntitled,nopponents,avg_dominance,tournament,first_game_dt,first_game_id
2,23,1: IM kenta123,19,-275.08695652173912549188,"11y0CwCg,TqH4KwYw",2017-07-01 05:54:01.476,zbbFV6ND
7,13,"4: FM Taipei101,LM Lance5500,CM Kingscrusher-Y...",12,-276.69230769230767918998,autumn17,2017-10-28 00:50:57.504,zoKB22Mr
62,20,"2: IM jsdbv,FM StewiieGriffin",14,-161.80000000000001136868,,2017-11-01 22:05:16.937,SZvqSUOL
81,26,"5: CM PchelkinVK,FM dragonwelb,IM lovlas,GM Ra...",19,-324.38461538461535837996,IKMZgOrd,2017-12-04 19:38:17.540,hlgiiNx6
148,13,,11,-214.38461538461538680167,,2018-01-12 19:21:06.743,qKS3rpax
167,17,"2: IM AutHun,CM akhairat",11,-279.17647058823530414884,,2018-01-25 13:01:23.459,qCvrSqAS
224,14,1: GM Morphy123,9,-277.78571428571427759380,,2018-02-15 16:01:44.544,qxXTNBlT
240,13,1: IM Tormozz,9,-257.15384615384613198330,,2018-02-24 11:33:53.601,4WWIV54v
253,13,1: IM damestar,7,-262.69230769230767918998,,2018-03-01 18:19:58.753,XNkcbcJS
295,14,,12,-372.28571428571427759380,IYjZF7dm,2018-04-04 18:47:32.982,vHBOMgF4


time: 16 ms (started: 2023-07-09 18:17:39 +03:00)


<IPython.core.display.Javascript object>

In [107]:
semenychev_streaks.query("nwins>=13")

,nwins,ntitled,nopponents,avg_dominance,tournament,first_game_dt,first_game_id
674,16,,15,-480.18750000000000000000,"46lBjsWk,rwUfWIqd",2021-05-23 22:06:57.830,TyiGxFWM
1061,14,1: GM Crest64,13,-74.50000000000000000000,"57XWjV8u,zIlkmSOR",2021-10-31 16:27:32.016,eEvyLX4D
1070,15,1: IM Agronom6428,13,-76.06666666666666287711,"fnlQRDmA,xbJQU5M5",2021-11-04 17:23:59.422,AAR9vHcE
1315,14,,14,-492.85714285714283278139,"UmbgVUL4,y9662BPk",2022-02-14 19:24:42.051,ALYQwgSO
1386,14,,13,-349.42857142857144481241,"Mp0K3Qpr,ahE4ba2h",2022-02-23 10:44:05.720,pI2OyE7L
1415,13,,13,-431.53846153846154720668,"HR3JtXQy,i2dvConF",2022-02-26 08:24:22.690,C9AN22Xf
1419,17,,16,-158.29411764705881182635,b4uBvRpD,2022-02-26 11:04:56.620,DcgdVl8N
1481,16,,15,-259.12500000000000000000,"5spL57pb,ZSqfXOSw,5XyL7NI3",2022-03-08 09:52:35.875,96IFIANI
1506,14,1: FM Karadenizaslani,14,-439.35714285714283278139,ZQNx9dtq,2022-03-19 12:00:45.778,U2Sx7cED
1511,14,,14,-470.21428571428572240620,"cPK6wTN3,5Q5Cv49O",2022-03-20 11:40:58.408,YneKwg53


time: 15 ms (started: 2023-07-09 18:17:42 +03:00)


<IPython.core.display.Javascript object>